In [5]:
import pandas as pd
import plotly.express as px

# Load your dataset (replace 'file_path' with your dataset path)
file_path = 'film_full_2.csv'
movies_df = pd.read_csv(file_path)

movies_df

C:\Users\wahid\AppData\Local\Temp\ipykernel_30944\2971480383.py:6: DtypeWarning:

Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.



,wikipedia_id,dataset_id,title,release_date,box_office,runtime,languages,countries,genres,reviewScores,awardsReceived,awardsNominated,capitalCost,release date,categories,nbOscarReceived,nbOscarNominated
0,29988427.0,/m/0cryrj0,!Women Art Revolution,2010-09-01,NaN,83.0,['English'],"['United States of America', 'Canada']","['LGBT', 'History', 'Documentary']",NaN,[],[],NaN,NaN,"['Documentary', 'Historical & Period', 'Other']",NaN,NaN
1,54213693.0,Q22123892,"""Hataraku Saibō!!"" Saikyō no Teki, Futatabi. K...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[''],[''],NaN,NaN,[''],NaN,NaN
2,72256712.0,Q115213551,"""Sr.""",2022-01-01,NaN,NaN,NaN,['United States of America'],[''],NaN,['Grierson Awards'],[''],NaN,2022,[''],NaN,NaN
3,30332673.0,/m/0crs0hx,#1 Cheerleader Camp,2010-01-01,NaN,90.0,NaN,['United States of America'],"['Sports', 'Sex comedy', 'Comedy film', 'Comed...",NaN,[],[],NaN,NaN,"['Drama', 'Sports', 'Comedy']",NaN,NaN
4,61446919.0,Q85740993,#Alive,2020-06-24,NaN,NaN,['Korean'],['South Korea'],"['zombie', 'apocalyptic']",0.77,[''],[''],NaN,2020,"['Horror', 'Action & Adventure']",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108955,12680019.0,/m/02x07kx,…All the Marbles,1981-01-01,NaN,113.0,['English'],['United States of America'],"['Drama', 'Comedy-drama', 'Comedy']",NaN,[],[],NaN,NaN,"['Drama', 'Comedy']",NaN,NaN
108956,27718784.0,/m/0cc744m,…and there wasn’t everafter…,NaN,NaN,21.0,['English'],['India'],['Short Film'],NaN,[],[],NaN,NaN,['Other'],NaN,NaN
108957,8277007.0,/m/026yqtg,…men Olsenbanden var ikke død,1984-01-01,NaN,89.0,['Norwegian'],['Norway'],"['Crime Fiction', 'Comedy']",NaN,[],[],NaN,NaN,"['Drama', 'Comedy']",NaN,NaN
108958,33021454.0,/m/0h54w6f,₤500 Reward,1918-01-01,NaN,NaN,"['Silent film', 'English']",NaN,['Silent film'],NaN,[],[],NaN,NaN,['Other'],NaN,NaN


In [6]:
from operator import contains


def get_num_awards(awards):
    if pd.isna(awards):
        return 0
    if len(awards) <= 4:
        return 0
    awards = awards.replace("[", "").replace("]", "").replace("'", "").split(",")
    return len(awards)

movies_df['nbOscarReceived'].value_counts()

movies_df['nbOscarReceived'] = movies_df['nbOscarReceived'].apply(lambda x: 0 if pd.isna(x) else int(x.split(" ")[0]) if isinstance(x, str) else x)
movies_df['nbOscarNominated'] = movies_df['nbOscarNominated'].apply(lambda x: 0 if pd.isna(x) else int(x.split("[")[0]) if isinstance(x, str) else x)

# movies_df['num_awards'] = movies_df['awardsNominated'].apply(get_num_awards) + movies_df['awardsReceived'].apply(get_num_awards) + movies_df['nbOscarReceived'] + movies_df['nbOscarNominated']
movies_df['num_awards'] = movies_df['awardsNominated'].apply(get_num_awards) + movies_df['nbOscarReceived']

In [7]:
from turtle import color


df_plot1 = movies_df[['box_office', 'num_awards', 'title', 'release_date']]
df_plot1 = df_plot1[df_plot1['num_awards'] > 0].dropna()
df_plot1['release_date'] = df_plot1['release_date'].apply(lambda x: x.split("-")[0])


fig1 = px.scatter(
    df_plot1,
    x='box_office',
    y='num_awards',
    title='Box Office Revenue vs. Awards Received',
    labels={'box_office': 'Box Office Revenue', 'awardsReceived': 'Awards Received'},
    color = 'release_date',
    log_x=True,  # Use log scale for box office
    hover_data=['title']
)
fig1.show()

df_plot2 = movies_df[['runtime', 'num_awards', 'title']]
df_plot2 = df_plot2[df_plot2['num_awards'] > 0].dropna()

fig2 = px.scatter(
    df_plot2,
    x='runtime',
    y='num_awards',
    title='Runtime vs. Awards Received',
    labels={'runtime': 'Runtime', 'awardsReceived': 'Awards Received'},
    hover_data=['title']
)

fig2.show()



df_plot3 = movies_df[['reviewScores', 'num_awards', 'title']]
df_plot3 = df_plot3[df_plot3['num_awards'] > 0].dropna()

fig3 = px.scatter(
    df_plot3,
    x='reviewScores',
    y='num_awards',
    title='Review Scores vs. Awards Received',
    labels={'reviewScores': 'Review Scores', 'awardsReceived': 'Awards Received'},
    hover_data=['title']
)

fig3.show()

In [8]:
import re


cleaned_data = movies_df.copy()
cleaned_data['categories'] = cleaned_data['categories'].apply(lambda x: x.replace("[", "").replace("]", "").replace("'", "").split(",") if isinstance(x, str) else x)
def strip(x):
    try :
        return [i.strip() for i in x]
    except:
        return x
cleaned_data['categories'] = cleaned_data['categories'].apply(strip)

In [9]:
import plotly.express as px

cat_awards = cleaned_data.explode('categories').groupby('categories')['num_awards'].sum().reset_index().sort_values('num_awards', ascending=False)

fig = px.bar(cat_awards,
              x='categories', y='num_awards', title="Award Nominations by Genre")
fig.show()

cleaned_data['release_month'] = cleaned_data['release_date'].apply(lambda x: x.split("-")[1] if isinstance(x, str) else x)
fig = px.line(cleaned_data.groupby('release_month')['num_awards'].sum().reset_index(),
              x='release_month', y='num_awards', 
              title="Award-Winning Movies by Release Month",
              labels={'release_month': 'Month', 'num_awards': 'Awards Received'})
fig.show()



In [10]:
cleaned_data['release_year'] = cleaned_data['release_date'].apply(lambda x: x.split("-")[0] if isinstance(x, str) else 0)
cleaned_data = cleaned_data[cleaned_data['release_year'].astype(int) >=2015]
fig = px.line(cleaned_data.groupby('release_year')['num_awards'].sum().reset_index(),
              x='release_year', y='num_awards', 
              title="Awards per Year",
              labels={'release_year': 'Year', 'num_awards': 'Awards Received'})
fig.show()
